<img src="https://cdn.discordapp.com/attachments/1111599839663370271/1123589551244460072/CatsVsDogs.jpg">

# **Transfer Learning & Fine Tuning on MobileNetV2**

---

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import datetime
import warnings
warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('dark_background')
from tqdm import tqdm_notebook

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
tf.test.is_gpu_available()

---

# Defining Classes & Generators

In [ ]:
classes = ['cats','dogs']

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.0, 
                               shear_range=0.15, 
                               zoom_range=0.2, 
                               horizontal_flip=True) 

train_data = generator.flow_from_directory( directory='../input/cat-and-dog/training_set/training_set', 
                                            target_size=(128, 128),
#                                             color_mode='grayscale',
                                            classes=classes, 
                                            batch_size=128)


test_data = generator.flow_from_directory( directory='../input/cat-and-dog/test_set/test_set', 
                                           target_size=(128, 128), 
#                                            color_mode='grayscale',
                                           classes=classes, 
                                           batch_size=128,
                                           shuffle=False)

---

# MobileNetV2

### Loading the Model

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),include_top=False,weights="imagenet")

In [ ]:
base_model.summary()

### Freezing the Model

In [ ]:
base_model.trainable = False

### Defining the Custom Head for network

In [ ]:
base_model.output

#### 2 Apporaches can be taken
1. Using Flattening Layer
2. Using GlobalAveragePooling Layer

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

In [ ]:
pred_layer = tf.keras.layers.Dense(units=2,activation="sigmoid")(global_average_layer)

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=pred_layer)
model.summary()

### Compiling the model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),loss="categorical_crossentropy",metrics=["accuracy"])

### Training the model

In [ ]:
earlyStopping =  tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy',restore_best_weights=True,start_from_epoch=2)

# history = model.fit_generator(train_data, epochs=5, validation_data=test_data,callbacks=[earlyStopping])
history = model.fit(train_data, epochs=10, validation_data=test_data, shuffle=True, callbacks=[earlyStopping],batch_size=64)

In [ ]:
df = pd.DataFrame(history.history)
df_loss = df[['loss', 'val_loss']]
df_acc =  df[['accuracy', 'val_accuracy']]

In [ ]:
plt.subplot(1,2,figsize=(10,4))
plt.title("Loss & Val Loss")
sns.lineplot(df_loss,palette="flare");

plt.title(1,2,"Accuracy & Val Accuracy")
plt.figure(figsize=(10,4))
sns.lineplot(df_acc,palette="flare");

### Fine-Tuning Base Model

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 